In [14]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold, apertureSize = 3)

def gaussian_blur(img, kernel_size, sigma = 0):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), sigma)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_line(img, line, color=[255, 0, 0], thickness=2):
    """Draw a single line using the rho and theta"""
    if isinstance(line, np.ndarray):
        a = np.cos(line[1])
        b = np.sin(line[1])
        x0 = a*line[0]
        y0 = b*line[0]
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img, (x1,y1), (x2,y2), (255,0,0), 8)

def hough_lines(img, rho, theta, threshold):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with the left and right lines drawn.
    """
    lines = cv2.HoughLines(img, rho, theta, threshold)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    #Split of the lines according to its slope
    linesBySide = getLinesBySide(lines)
    
    #Getting the mean of all left lines 
    leftLine = np.mean(linesBySide[0], axis=0)
    
    #Getting the mean of all right lines 
    rightLine = np.mean(linesBySide[1], axis=0)
    
    #Draw lines 
    draw_line(line_img, leftLine)
    draw_line(line_img, rightLine)
        
    return line_img


# Python 3 has support for cool math symbols.
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def getLinesBySide(lines):
    results = [[], []]
    if isinstance(lines, np.ndarray):
        for line in lines:
            for rho,theta in line:
                t = -1 * math.cos(theta)
                m = t / math.sin(theta)
                if m > 0:
                    results[0].append([rho,theta])
                else:
                    results[1].append([rho,theta])
    return results


# Pipeline that will draw lane lines on the frame
def process_image(image):
    #Image converted to gray scale
    gray = grayscale(image)
    
    #The image is blur to get rip of edges that I dont care about 
    blur_gray = gaussian_blur(gray, 11, 2)
    
    #Get edges
    edges = canny(blur_gray, 50, 150)
    
    #Using the image shape I get the vertices to isolate the area with the lane lines.
    imshape = image.shape
    vertices = np.array([[(imshape[1] * 0.15,imshape[0]),(450, imshape[0] * 0.6), (540, imshape[0] * 0.6), (imshape[1] * 0.95,imshape[0])]], dtype=np.int32)
    
    #Get the specific area with the edges that I care about.
    masked_edges = region_of_interest(edges, vertices)
    
    #Get an image with the left and right lines.
    line_img = hough_lines(masked_edges,1,np.pi/180, 35)
    
    #Again, using the same vertices I limit the size of the right and left lines
    masked_line_img = region_of_interest(line_img, vertices)
    
    #I get the combination of the original image with the image that only contains the left and right lines.
    resultImage = weighted_img(masked_line_img, image, α=0.8, β=1., λ=0.)
    
    return resultImage

In [15]:
yellow_output = 'yellow.mp4'

clip1 = VideoFileClip("solidYellowLeft.mp4")

yellow_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:11<00:00, 60.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 11.6 s


In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [38]:
white_output = 'white.mp4'

clip1 = VideoFileClip("solidWhiteRight.mp4")

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:05<00:00, 20.91it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 5.67 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [10]:

challenge_output = 'challengeResult.mp4'

clip1 = VideoFileClip("challenge.mp4")

challenge_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video challengeResult.mp4
[MoviePy] Writing video challengeResult.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:07<00:00, 33.23it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challengeResult.mp4 

Wall time: 8.23 s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))